In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


c:\Users\sagam\miniconda3\envs\aquasense\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
df = pd.read_csv("../data/final/master_dataset.csv")
df["dateTime"] = pd.to_datetime(df["dateTime"])
df.set_index("dateTime", inplace=True)


In [3]:
X = df[["TN", "NH3", "NO23", "OP", "SSC"]]
y = df["TP"]


In [4]:
from sklearn.preprocessing import MinMaxScaler

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1,1))


In [5]:
def create_sequences(X, y, lookback=5):
    Xs, ys = [], []
    for i in range(len(X) - lookback):
        Xs.append(X[i:i+lookback])
        ys.append(y[i+lookback])
    return np.array(Xs), np.array(ys)

LOOKBACK = 5
X_seq, y_seq = create_sequences(X_scaled, y_scaled, LOOKBACK)


In [6]:
split = int(0.8 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(32, input_shape=(LOOKBACK, X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")


c:\Users\sagam\miniconda3\envs\aquasense\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.2,
    verbose=1
)


Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - loss: 0.0535 - val_loss: 0.0168
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0297 - val_loss: 0.0202
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0276 - val_loss: 0.0174
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0269 - val_loss: 0.0182
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0266 - val_loss: 0.0168
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0265 - val_loss: 0.0164
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0261 - val_loss: 0.0163
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0259 - val_loss: 0.0159
Epoch 9/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0259 - val_loss: 0.0157
Epoch 10/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0257 - val_loss: 0.0154
Epoch 11/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0257 - val_loss: 0.0165
Epoch 12/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.02

In [9]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test)

y_test_inv = scaler_y.inverse_transform(y_test)
y_pred_inv = scaler_y.inverse_transform(y_pred)

print("RMSE:", mean_squared_error(y_test_inv, y_pred_inv, squared=False))
print("R2:", r2_score(y_test_inv, y_pred_inv))


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step
RMSE: 0.06880589664953916
R2: -0.054199997190134575


c:\Users\sagam\miniconda3\envs\aquasense\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
